In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import string
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelWithLMHead

In [ ]:
# Constants

DEVICE = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {DEVICE}")

Using device: cuda


## Data Loading

In [ ]:
# https://huggingface.co/datasets/google/Synthetic-Persona-Chat
splits = {'train': 'data/Synthetic-Persona-Chat_train.csv', 'validation': 'data/Synthetic-Persona-Chat_valid.csv', 'test': 'data/Synthetic-Persona-Chat_test.csv'}

In [ ]:
df_train = pd.read_csv("hf://datasets/google/Synthetic-Persona-Chat/" + splits["train"])
df_val = pd.read_csv("hf://datasets/google/Synthetic-Persona-Chat/" + splits["validation"])
df_test = pd.read_csv("hf://datasets/google/Synthetic-Persona-Chat/" + splits["test"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(8938, 3)
(1000, 3)
(968, 3)


In [ ]:
df_train.head()

,user 1 personas,user 2 personas,Best Generated Conversation
0,I am 32.\nI do not want a job.\nI play video g...,My favorite drink is iced coffee.\nI have a bl...,User 1: Hi! I'm [user 1's name].\nUser 2: Hi [...
1,I am 32.\nI play video games all day.\nI still...,I have a ford f150.\nI like ford cars.\nMy tru...,"User 1: Hey, how's it going?\nUser 2: Good, I'..."
2,I am 32.\nI play video games all day.\nI still...,I can recite the movie young frankenstein word...,"User 1: Hi, my name is John. What's your name?..."
3,I write.\nI work at mcdonald s.\nI watch youtu...,I want to move.\nI don t like feeling controll...,User 1: Hi!\nUser 2: Hey!\nUser 1: What's up?\...
4,I am bald.\nI like to swim.\nMy favorite drink...,My favorite store is american eagle.\nI enjoy ...,User 1: Hello!\nUser 2: Hi!\nUser 1: What do y...


In [ ]:
df_train_convo = df_train["Best Generated Conversation"]
df_train_convo.head()
df_train_convo[0]

"User 1: Hi! I'm [user 1's name].\nUser 2: Hi [user 1's name], I'm [user 2's name].\nUser 1: What do you do for fun?\nUser 2: I like to play video games, go to the beach, and read.\nUser 1: I like to play video games too! I'm not much of a reader, though.\nUser 2: What video games do you like to play?\nUser 1: I like to play a lot of different games, but I'm really into competitive online games right now.\nUser 2: I'm not really into competitive games, I like to play more relaxing games.\nUser 1: That's cool. What kind of relaxing games do you like to play?\nUser 2: I like to play puzzle games, simulation games, and story-based games.\nUser 1: I've never been much of a puzzle game person, but I do like simulation games and story-based games.\nUser 2: Nice! What's your favorite simulation game?\nUser 1: I like Stardew Valley a lot. It's a farming game, but it's also really relaxing and fun.\nUser 2: I've heard good things about that game. I might have to check it out.\nUser 1: You shoul

### Pytorch dataset and dataloader

In [ ]:
class SarcasmDataset(Dataset):
    def __init__(self, conversations):
        self.conversations = conversations

    def __len__(self):
        return len(self.conversations)

    def __getitem__(self, idx):
        return self.conversations[idx]

dataset_train_convo = SarcasmDataset(df_train_convo)

In [ ]:
# Define DataLoader parameters
batch_size = 32  # Adjust based on your hardware capabilities
num_workers = 2  # Number of subprocesses for data loading

# Initialize DataLoader
train_loader = DataLoader(dataset_train_convo, batch_size=batch_size, shuffle=False, num_workers=num_workers)


## Sarcasm Detection HF Model 1
https://huggingface.co/helinivan/english-sarcasm-detector

In [ ]:
def preprocess_data(text: str) -> str:
   return text.lower().translate(str.maketrans("", "", string.punctuation)).strip()


In [ ]:
MODEL_PATH_1 = "helinivan/english-sarcasm-detector"

tokenizer_1 = AutoTokenizer.from_pretrained(MODEL_PATH_1)
model_1 = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH_1)

# Set device to GPU if available
model_1.to(DEVICE)
model_1.eval()  # Set model to evaluation model

tokenizer_config.json:   0%|          | 0.00/400 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# text = "CIA Realizes It's Been Using Black Highlighters All These Years."
text = df_train_convo[0]
tokenized_text = tokenizer_1([preprocess_data(text)], padding=True, truncation=True, max_length=256, return_tensors="pt")
output = model_1(**tokenized_text.to(DEVICE))
probs = output.logits.softmax(dim=-1).tolist()[0]
confidence = max(probs)
prediction = probs.index(confidence)

results = {"is_sarcastic": prediction, "confidence": confidence}


In [ ]:
results

{'is_sarcastic': 1, 'confidence': 0.6577213406562805}

In [ ]:
# Initialize lists to store results
is_sarcastic_list_1 = []
confidence_list_1 = []
results_list_1 = []

# Disable gradient calculations for inference
with torch.no_grad():
    for batch in tqdm(train_loader, desc="Processing Batches"):
        # Preprocess the batch of conversations
        preprocessed_batch = [preprocess_data(convo) for convo in batch]

        # Tokenize the batch
        tokenized_batch = tokenizer_1(
            preprocessed_batch,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )

        # Move tensors to the configured device
        input_ids = tokenized_batch["input_ids"].to(DEVICE)
        attention_mask = tokenized_batch["attention_mask"].to(DEVICE)

        # Forward pass through the model
        output = model_1(input_ids=input_ids, attention_mask=attention_mask)

        # Compute softmax probabilities
        probs = torch.softmax(output.logits, dim=-1).cpu().tolist()

        # Extract predictions and confidences
        for prob in probs:
            confidence = max(prob)
            prediction = prob.index(confidence)
            results = {"is_sarcastic": prediction, "confidence": confidence}
            results_list_1.append(results)
            is_sarcastic_list_1.append(prediction)
            confidence_list_1.append(confidence)

Processing Batches: 100%|██████████| 280/280 [04:39<00:00,  1.00it/s]


In [ ]:
print(len(results_list_1))
print(len(is_sarcastic_list_1))
print(len(confidence_list_1))

8938
8938
8938


In [ ]:
# Get the analysis results into the df_train
df_train[MODEL_PATH_1 + ": is_sarcastic"] = is_sarcastic_list_1
df_train[MODEL_PATH_1 + ": confidence"] = confidence_list_1

df_train.head()

,user 1 personas,user 2 personas,Best Generated Conversation,helinivan/english-sarcasm-detector: is_sarcastic,helinivan/english-sarcasm-detector: confidence
0,I am 32.\nI do not want a job.\nI play video g...,My favorite drink is iced coffee.\nI have a bl...,User 1: Hi! I'm [user 1's name].\nUser 2: Hi [...,1,0.657721
1,I am 32.\nI play video games all day.\nI still...,I have a ford f150.\nI like ford cars.\nMy tru...,"User 1: Hey, how's it going?\nUser 2: Good, I'...",0,0.663435
2,I am 32.\nI play video games all day.\nI still...,I can recite the movie young frankenstein word...,"User 1: Hi, my name is John. What's your name?...",0,0.956261
3,I write.\nI work at mcdonald s.\nI watch youtu...,I want to move.\nI don t like feeling controll...,User 1: Hi!\nUser 2: Hey!\nUser 1: What's up?\...,0,0.747205
4,I am bald.\nI like to swim.\nMy favorite drink...,My favorite store is american eagle.\nI enjoy ...,User 1: Hello!\nUser 2: Hi!\nUser 1: What do y...,0,0.806041


### Inspection

In [ ]:
# Optional: Inspecting Sarcastic Entries for Model 1

indices_of_1 = []

for index, entry in enumerate(results_list_1):
    if entry['is_sarcastic'] == 1:
        indices_of_1.append(index)

sarcastic_rows_1 = df_train_convo.iloc[indices_of_1]
sarcastic_rows_1.head()

print(sarcastic_rows_1.iloc[2])  # Row where sarcasm is detected

User 1: I'm not very social either. I find it easier to talk to people online than in person.
User 2: I find talking to people in person to be boring. I prefer to keep to myself.
User 1: I think it's nice to be able to socialize with people, but I understand why you prefer to be alone.
User 2: I like to be in control of my environment, and when I'm alone, I can do whatever I want without having to worry about anyone else.
User 1: That makes sense. I think it's important to have some time to yourself each day, even if you're an extrovert.
User 2: I agree. It's important to be able to relax and recharge.
User 1: Do you have any hobbies?
User 2: I like to read, write, and play video games.
User 1: Me too! I love to read and play video games.
User 2: What kind of games do you like to play?
User 1: I like to play all kinds of games, but I especially like RPGs.
User 2: Me too! I love the story and character development in RPGs.
User 1: I agree. They're really immersive and you can get really

## Sarcasm Detection HF Model 2
https://huggingface.co/dima806/sarcasm-detection-distilbert?library=transformers - newest model

In [ ]:
# Model path for Model 2
MODEL_PATH_2 = "dima806/sarcasm-detection-distilbert"

# Initialize tokenizer and model for Model 2
tokenizer_2 = AutoTokenizer.from_pretrained(MODEL_PATH_2)
model_2 = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH_2)

# Set device and model to eval
model_2.to(DEVICE)
model_2.eval()

config.json:   0%|          | 0.00/800 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# Initialize lists to store results for Model 2
is_sarcastic_list_2 = []
confidence_list_2 = []
results_list_2 = []

# Disable gradient calculations for inference
with torch.no_grad():
    for batch in tqdm(train_loader, desc="Processing Batches for Model 2"):
        # Preprocess the batch of conversations
        preprocessed_batch = [preprocess_data(convo) for convo in batch]

        # Tokenize the batch using Model 2's tokenizer
        tokenized_batch = tokenizer_2(
            preprocessed_batch,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )

        # Move tensors to the configured device
        input_ids = tokenized_batch["input_ids"].to(DEVICE)
        attention_mask = tokenized_batch["attention_mask"].to(DEVICE)

        # Forward pass through Model 2
        output = model_2(input_ids=input_ids, attention_mask=attention_mask)

        # Compute softmax probabilities
        probs = torch.softmax(output.logits, dim=-1).cpu().tolist()

        # Extract predictions and confidences
        for prob in probs:
            confidence = max(prob)
            prediction = prob.index(confidence)
            results = {"is_sarcastic": prediction, "confidence": confidence}
            results_list_2.append(results)
            is_sarcastic_list_2.append(prediction)
            confidence_list_2.append(confidence)

Processing Batches for Model 2: 100%|██████████| 280/280 [04:43<00:00,  1.01s/it]


In [ ]:
# Add results to df_train for Model 2
df_train[f"{MODEL_PATH_2}: is_sarcastic"] = is_sarcastic_list_2
df_train[f"{MODEL_PATH_2}: confidence"] = confidence_list_2

df_train.head()

,user 1 personas,user 2 personas,Best Generated Conversation,helinivan/english-sarcasm-detector: is_sarcastic,helinivan/english-sarcasm-detector: confidence,dima806/sarcasm-detection-distilbert: is_sarcastic,dima806/sarcasm-detection-distilbert: confidence
0,I am 32.\nI do not want a job.\nI play video g...,My favorite drink is iced coffee.\nI have a bl...,User 1: Hi! I'm [user 1's name].\nUser 2: Hi [...,1,0.657721,0,0.980763
1,I am 32.\nI play video games all day.\nI still...,I have a ford f150.\nI like ford cars.\nMy tru...,"User 1: Hey, how's it going?\nUser 2: Good, I'...",0,0.663435,0,0.966272
2,I am 32.\nI play video games all day.\nI still...,I can recite the movie young frankenstein word...,"User 1: Hi, my name is John. What's your name?...",0,0.956261,0,0.953414
3,I write.\nI work at mcdonald s.\nI watch youtu...,I want to move.\nI don t like feeling controll...,User 1: Hi!\nUser 2: Hey!\nUser 1: What's up?\...,0,0.747205,0,0.910250
4,I am bald.\nI like to swim.\nMy favorite drink...,My favorite store is american eagle.\nI enjoy ...,User 1: Hello!\nUser 2: Hi!\nUser 1: What do y...,0,0.806041,0,0.911955


### Inspection

In [ ]:
# Aggregating the results for Model 2
counts_2 = df_train[f"{MODEL_PATH_2}: is_sarcastic"].value_counts().to_dict()
sums_2 = df_train.groupby(f"{MODEL_PATH_2}: is_sarcastic")[f"{MODEL_PATH_2}: confidence"].sum().to_dict()

averages_2 = {label: (sums_2[label] / counts_2[label] if counts_2[label] > 0 else 0) for label in counts_2}

print("Model 2 - Occurrences:", counts_2)
print("Model 2 - Average Confidence:", averages_2)

Model 2 - Occurrences: {0: 8922, 1: 16}
Model 2 - Average Confidence: {0: 0.9455389274566918, 1: 0.6172291487455368}


In [ ]:
# Optional: Inspecting Sarcastic Entries for Model 2

indices_of_2 = []

for index, entry in enumerate(results_list_2):
    if entry['is_sarcastic'] == 1:
        indices_of_2.append(index)

sarcastic_rows_2 = df_train_convo.iloc[indices_of_2]
sarcastic_rows_2.head()

print(sarcastic_rows_2.iloc[2])  # Row where sarcasm is detected

User 1: What's your favorite food?
User 2: Meatloaf!
User 1: That's interesting, I've never really liked meatloaf.
User 2: Oh, it's so good! My grandma used to make it for me all the time when I was little.
User 1: That's sweet. I'm not really a big fan of cooking, but I do enjoy eating.
User 2: Me too! I love to cook, but I'm not very good at it.
User 1: Well, you're better than me! I don't even know how to boil water.
User 2: That's okay, everyone has to start somewhere.
User 1: I guess so. But I'm not sure I'll ever be a good cook.
User 2: You never know! You might surprise yourself.
User 1: I hope so. Maybe I'll try making meatloaf one day.
User 2: That would be awesome! I'd love to try your meatloaf.
User 1: I'll let you know how it goes.
User 2: Great! I'm excited to try it.
User 1: Me too!
User 2: So, what else do you like to do?
User 1: I like to go shopping with my daughters.
User 2: That sounds fun! I don't have any kids, but I like to go shopping with my friends.
User 1: It'

## Sarcasm Detection HF Model 3
https://huggingface.co/jkhan447/sarcasm-detection-Bert-base-uncased-newdata

In [ ]:
# Model path for Model 3
MODEL_PATH_3 = "jkhan447/sarcasm-detection-Bert-base-uncased-newdata"

# Initialize tokenizer and model for Model 3
tokenizer_3 = AutoTokenizer.from_pretrained(MODEL_PATH_3)
model_3 = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH_3)

# Set device and model to eval
model_3.to(DEVICE)
model_3.eval()

tokenizer_config.json:   0%|          | 0.00/348 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# Initialize lists to store results for Model 3
is_sarcastic_list_3 = []
confidence_list_3 = []
results_list_3 = []

# Disable gradient calculations for inference
with torch.no_grad():
    for batch in tqdm(train_loader, desc="Processing Batches for Model 3"):
        # Preprocess the batch of conversations
        preprocessed_batch = [preprocess_data(convo) for convo in batch]

        # Tokenize the batch using Model 3's tokenizer
        tokenized_batch = tokenizer_3(
            preprocessed_batch,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )

        # Move tensors to the configured device
        input_ids = tokenized_batch["input_ids"].to(DEVICE)
        attention_mask = tokenized_batch["attention_mask"].to(DEVICE)

        # Forward pass through Model 3
        output = model_3(input_ids=input_ids, attention_mask=attention_mask)

        # Compute softmax probabilities
        probs = torch.softmax(output.logits, dim=-1).cpu().tolist()

        # Extract predictions and confidences
        for prob in probs:
            confidence = max(prob)
            prediction = prob.index(confidence)
            results = {"is_sarcastic": prediction, "confidence": confidence}
            results_list_3.append(results)
            is_sarcastic_list_3.append(prediction)
            confidence_list_3.append(confidence)

Processing Batches for Model 3: 100%|██████████| 280/280 [04:44<00:00,  1.02s/it]


In [ ]:
# Add results to df_train for Model 3
df_train[f"{MODEL_PATH_3}: is_sarcastic"] = is_sarcastic_list_3
df_train[f"{MODEL_PATH_3}: confidence"] = confidence_list_3

df_train.head()

,user 1 personas,user 2 personas,Best Generated Conversation,helinivan/english-sarcasm-detector: is_sarcastic,helinivan/english-sarcasm-detector: confidence,dima806/sarcasm-detection-distilbert: is_sarcastic,dima806/sarcasm-detection-distilbert: confidence,dnzblgn/Sarcasm-Detection-Customer-Reviews: is_sarcastic,dnzblgn/Sarcasm-Detection-Customer-Reviews: confidence,jkhan447/sarcasm-detection-Bert-base-uncased-newdata: is_sarcastic,jkhan447/sarcasm-detection-Bert-base-uncased-newdata: confidence
0,I am 32.\nI do not want a job.\nI play video g...,My favorite drink is iced coffee.\nI have a bl...,User 1: Hi! I'm [user 1's name].\nUser 2: Hi [...,1,0.657721,0,0.980763,0,1.0,0,0.990719
1,I am 32.\nI play video games all day.\nI still...,I have a ford f150.\nI like ford cars.\nMy tru...,"User 1: Hey, how's it going?\nUser 2: Good, I'...",0,0.663435,0,0.966272,0,1.0,0,0.929502
2,I am 32.\nI play video games all day.\nI still...,I can recite the movie young frankenstein word...,"User 1: Hi, my name is John. What's your name?...",0,0.956261,0,0.953414,0,1.0,0,0.990292
3,I write.\nI work at mcdonald s.\nI watch youtu...,I want to move.\nI don t like feeling controll...,User 1: Hi!\nUser 2: Hey!\nUser 1: What's up?\...,0,0.747205,0,0.910250,0,1.0,0,0.923233
4,I am bald.\nI like to swim.\nMy favorite drink...,My favorite store is american eagle.\nI enjoy ...,User 1: Hello!\nUser 2: Hi!\nUser 1: What do y...,0,0.806041,0,0.911955,0,1.0,0,0.986866


### Inspection

In [ ]:
# Aggregating the results for Model 3
counts_3 = df_train[f"{MODEL_PATH_3}: is_sarcastic"].value_counts().to_dict()
sums_3 = df_train.groupby(f"{MODEL_PATH_3}: is_sarcastic")[f"{MODEL_PATH_3}: confidence"].sum().to_dict()

averages_3 = {label: (sums_3[label] / counts_3[label] if counts_3[label] > 0 else 0) for label in counts_3}

print("Model 3 - Occurrences:", counts_3)
print("Model 3 - Average Confidence:", averages_3)

Model 3 - Occurrences: {0: 8883, 1: 55}
Model 3 - Average Confidence: {0: 0.9786529140967339, 1: 0.6957050616090948}


In [ ]:
# Optional: Inspecting Sarcastic Entries for Model 3

indices_of_3 = []

for index, entry in enumerate(results_list_3):
    if entry['is_sarcastic'] == 1:
        indices_of_3.append(index)

sarcastic_rows_3 = df_train_convo.iloc[indices_of_3]
sarcastic_rows_3.head()

print(sarcastic_rows_3.iloc[2])  # Row where sarcasm is detected

User 1: Hi, I'm (name).
User 2: Hi, I'm (name).
User 1: What do you like to do for fun?
User 2: I love to skate and ski.
User 1: That's cool, I love to run.
User 2: Me too! I run every day to stay in shape.
User 1: I used to run every day, but I got a little lazy.
User 2: You should start running again! It's a great way to stay healthy and relieve stress.
User 1: I know, I know. I'll start again soon.
User 2: Good! I'm glad to hear that.
User 1: So, what do you do for work?
User 2: I'm a lawyer.
User 1: Oh, really? That must be interesting.
User 2: It is. I love working with people and helping them solve their problems.
User 1: That's great. I'm a lawyer too.
User 2: Really? That's awesome! What kind of law do you practice?
User 1: I'm a family law attorney.
User 2: Oh, that must be interesting. I've never met a family law attorney before.
User 1: It is. I love helping families through difficult times.
User 2: That's great. I'm glad you're passionate about your work.
User 1: Thanks. I 

## Aggregate model outputs

In [ ]:
# First we save the dataframe for future use
df_train.to_csv("df_train.csv", index=False)

### Majority voting

In [ ]:
# Define the is_sarcastic columns from each model
is_sarcastic_columns = [
    f"{MODEL_PATH_1}: is_sarcastic",
    f"{MODEL_PATH_2}: is_sarcastic",
    f"{MODEL_PATH_3}: is_sarcastic"
]

# Perform majority voting
df_train['majority_vote_sarcastic'] = df_train[is_sarcastic_columns].sum(axis=1).apply(lambda x: 1 if x >=2 else 0)

# Display the first few entries to verify
print(df_train[['majority_vote_sarcastic']].head())

   majority_vote_sarcastic
0                        0
1                        0
2                        0
3                        0
4                        0


### Average probability

In [ ]:
# Define the is_sarcastic and confidence columns from each model
models = [
    {
        "is_sarcastic": f"{MODEL_PATH_1}: is_sarcastic",
        "confidence": f"{MODEL_PATH_1}: confidence"
    },
    {
        "is_sarcastic": f"{MODEL_PATH_2}: is_sarcastic",
        "confidence": f"{MODEL_PATH_2}: confidence"
    },
    {
        "is_sarcastic": f"{MODEL_PATH_3}: is_sarcastic",
        "confidence": f"{MODEL_PATH_3}: confidence"
    }
]

# Function to compute probability of being sarcastic based on model's prediction
def compute_sarcastic_prob(row, model):
    if row[model["is_sarcastic"]] == 1:
        return row[model["confidence"]]
    else:
        return 1 - row[model["confidence"]]

# Apply the function to each model and store the probabilities
for idx, model in enumerate(models, 1):
    prob_col = f"model_{idx}_prob_sarcastic"
    df_train[prob_col] = df_train.apply(lambda row: compute_sarcastic_prob(row, model), axis=1)

# Now, compute the average probability across all models
prob_sarcastic_columns = [f"model_{idx}_prob_sarcastic" for idx in range(1, len(models)+1)]
df_train['average_confidence_sarcastic'] = df_train[prob_sarcastic_columns].mean(axis=1)

# Display the first few entries to verify
print(df_train[['average_confidence_sarcastic']].head())

   average_confidence_sarcastic
0                      0.228746
1                      0.146930
2                      0.033344
3                      0.139771
4                      0.098380


### Aggregation Results

In [ ]:
df_train.head()

,user 1 personas,user 2 personas,Best Generated Conversation,helinivan/english-sarcasm-detector: is_sarcastic,helinivan/english-sarcasm-detector: confidence,dima806/sarcasm-detection-distilbert: is_sarcastic,dima806/sarcasm-detection-distilbert: confidence,jkhan447/sarcasm-detection-Bert-base-uncased-newdata: is_sarcastic,jkhan447/sarcasm-detection-Bert-base-uncased-newdata: confidence,majority_vote_sarcastic,model_1_prob_sarcastic,model_2_prob_sarcastic,model_3_prob_sarcastic,average_confidence_sarcastic
0,I am 32.\nI do not want a job.\nI play video g...,My favorite drink is iced coffee.\nI have a bl...,User 1: Hi! I'm [user 1's name].\nUser 2: Hi [...,1,0.657721,0,0.980763,0,0.990719,0,0.657721,0.019237,0.009281,0.228746
1,I am 32.\nI play video games all day.\nI still...,I have a ford f150.\nI like ford cars.\nMy tru...,"User 1: Hey, how's it going?\nUser 2: Good, I'...",0,0.663435,0,0.966272,0,0.929502,0,0.336565,0.033728,0.070498,0.146930
2,I am 32.\nI play video games all day.\nI still...,I can recite the movie young frankenstein word...,"User 1: Hi, my name is John. What's your name?...",0,0.956261,0,0.953414,0,0.990292,0,0.043739,0.046586,0.009708,0.033344
3,I write.\nI work at mcdonald s.\nI watch youtu...,I want to move.\nI don t like feeling controll...,User 1: Hi!\nUser 2: Hey!\nUser 1: What's up?\...,0,0.747205,0,0.910250,0,0.923233,0,0.252795,0.089750,0.076767,0.139771
4,I am bald.\nI like to swim.\nMy favorite drink...,My favorite store is american eagle.\nI enjoy ...,User 1: Hello!\nUser 2: Hi!\nUser 1: What do y...,0,0.806041,0,0.911955,0,0.986866,0,0.193959,0.088045,0.013134,0.098380


In [ ]:
# Aggregated Statistics
print("\n=== Aggregated Statistics ===\n")

# 1. Majority Voting Statistics
print("Majority Voting - Occurrences:")
df_train['majority_vote_sarcastic'].value_counts()


=== Aggregated Statistics ===

Majority Voting - Occurrences:


,count
majority_vote_sarcastic,
0,8921
1,17


In [ ]:
print("\nMajority Voting - Average Confidence:")
# For average confidence in majority voting, compute the mean confidence where majority_vote_sarcastic ==1
avg_conf_majority = df_train[df_train['majority_vote_sarcastic'] ==1]['average_confidence_sarcastic'].mean()
print(f"Average Confidence for Sarcastic (Majority Vote): {avg_conf_majority:.4f}")


Majority Voting - Average Confidence:
Average Confidence for Sarcastic (Majority Vote): 0.4663


In [ ]:
# 2. Average Confidence Statistics
print("\nAverage Confidence - Statistics:")
df_train['average_confidence_sarcastic'].describe()


Average Confidence - Statistics:


,average_confidence_sarcastic
count,8938.000000
mean,0.122702
std,0.063167
min,0.018864
25%,0.074581
50%,0.111979
75%,0.160040
max,0.642201


In [ ]:
# display some results of the majority voted sarcastic entries
samples = df_train[df_train['majority_vote_sarcastic'] ==1].head().reset_index(drop=True)

for i, sample in samples.iterrows():
    print(f"=== Sample Entry {i} ===\n{sample['Best Generated Conversation']}\n**Average Confidence**: {sample['average_confidence_sarcastic']}\n\n")

=== Sample Entry 0 ===
User 1: Hi, I'm (name).
User 2: Hi, I'm (name).
User 1: What do you like to do for fun?
User 2: I love to skate and ski.
User 1: That's cool, I love to run.
User 2: Me too! I run every day to stay in shape.
User 1: I used to run every day, but I got a little lazy.
User 2: You should start running again! It's a great way to stay healthy and relieve stress.
User 1: I know, I know. I'll start again soon.
User 2: Good! I'm glad to hear that.
User 1: So, what do you do for work?
User 2: I'm a lawyer.
User 1: Oh, really? That must be interesting.
User 2: It is. I love working with people and helping them solve their problems.
User 1: That's great. I'm a lawyer too.
User 2: Really? That's awesome! What kind of law do you practice?
User 1: I'm a family law attorney.
User 2: Oh, that must be interesting. I've never met a family law attorney before.
User 1: It is. I love helping families through difficult times.
User 2: That's great. I'm glad you're passionate about your w